#### ***ML_Modules.py***

In [ ]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    accuracy_score,
    recall_score,
    f1_score,
    precision_score,
)

def data_scale(input_df):
    scaler = StandardScaler()
    scaled = scaler.fit_transform(input_df)

    # Return scaled data as DataFrame with same columns and index
    return pd.DataFrame(
        scaled,
        columns=input_df.columns,
        index=input_df.index
    )

def evaluate_classifier(y_test, y_pred):
    cm = confusion_matrix(y_test, y_pred)
    print("Confusion Matrix")
    print(cm)
    print("===================\n")

    print("Classification Report:")
    print(classification_report(y_test, y_pred, digits=3))
    print("===================")

    acc = accuracy_score(y_test, y_pred)
    rec = recall_score(y_test, y_pred, pos_label=1)
    f1 = f1_score(y_test, y_pred, pos_label=1)
    prec = precision_score(y_test, y_pred, pos_label=1)

    print(f"accuracy: {acc:.3f}")
    print(f"recall: {rec:.3f}")
    print(f"f1-score: {f1:.3f}")
    print(f"precision: {prec:.3f}")


#### ***Main.py***

In [ ]:
import os
import sys
import warnings
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.svm import SVC
import ML_Modules as mm

warnings.simplefilter(action="ignore")


def main():
    filename = input().strip()
    file_path = os.path.join(sys.path[0], filename)

    try:
        df = pd.read_csv(file_path)
    except FileNotFoundError:
        print(f"Error: File '{filename}' not found.")
        sys.exit(1)

    if "left" not in df.columns:
        print("Error: 'left' column not found.")
        sys.exit(1)

    le_salary = LabelEncoder()
    df["salary.enc"] = le_salary.fit_transform(df["salary"])
    le_dept = LabelEncoder()
    df["Department.enc"] = le_dept.fit_transform(df["Department"])

    df = df.drop(columns=["salary", "Department"])

    y = df["left"]
    X = df.drop(columns=["left"])

    X_scaled = mm.data_scale(X)

    X_train, X_test, y_train, y_test = train_test_split(
        X_scaled, y, test_size=0.20, random_state=42
    )

    param_grid = {
        "C": [0.1, 1, 10, 100],
        "gamma": ["scale", "auto", 0.01, 0.1, 1],
        "kernel": ["rbf"],
    }
    base_svc = SVC()
    grid = GridSearchCV(
        estimator=base_svc,
        param_grid=param_grid,
        cv=5,
        scoring="accuracy",
        n_jobs=-1,
    )
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_

    y_pred = best_model.predict(X_test)

    mm.evaluate_classifier(y_test, y_pred)


main()
